<a href="https://www.kaggle.com/code/hosseintimasi/energy-forecasting-data-challenge?scriptVersionId=162318186" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Import libraries and loading packages

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Models from sklearn library to use in prediction
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
# Mpdel from sklearn library to split the dataset
from sklearn.model_selection import train_test_split
# Model from sklearn library to gain the validation score using mean absolut error
from sklearn.metrics import mean_absolute_error


In [2]:
# importing the data sets
sample_sub = pd.read_csv('/kaggle/input/energy-forecasting-data-challenge-public/sample_submission.csv')
train = pd.read_csv('/kaggle/input/energy-forecasting-data-challenge-public/train.csv')
test = pd.read_csv('/kaggle/input/energy-forecasting-data-challenge-public/test.csv')

In [3]:
# checkin the train dataset
train

,time,P,Gb(i),Gd(i),H_sun,T2m,WS10m,load,residual_load,dataset_id
0,2018-01-01 00:00:00,0.0,0.0,0.0,0.0,9.11,5.38,113.5,113.5,1
1,2018-01-01 00:15:00,0.0,0.0,0.0,0.0,8.89,5.43,119.0,119.0,1
2,2018-01-01 00:30:00,0.0,0.0,0.0,0.0,8.66,5.48,117.5,117.5,1
3,2018-01-01 00:45:00,0.0,0.0,0.0,0.0,8.44,5.54,114.5,114.5,1
4,2018-01-01 01:00:00,0.0,0.0,0.0,0.0,8.22,5.59,123.0,123.0,1
...,...,...,...,...,...,...,...,...,...,...
88707,2020-10-10 22:45:00,0.0,0.0,0.0,0.0,6.49,2.83,99.5,99.5,11
88708,2020-10-10 23:00:00,0.0,0.0,0.0,0.0,6.39,2.83,96.5,96.5,11
88709,2020-10-10 23:15:00,0.0,0.0,0.0,0.0,6.34,2.83,100.5,100.5,11
88710,2020-10-10 23:30:00,0.0,0.0,0.0,0.0,6.30,2.83,97.5,97.5,11


In [4]:
train.groupby('dataset_id').describe()

P                                                      \
             count        mean         std  min  25%     50%       75%   
dataset_id                                                               
1           8064.0   52.195671  103.351092  0.0  0.0   0.000   55.8000   
2           8064.0  148.965188  187.977373  0.0  0.0  41.520  278.8050   
3           8064.0  140.732636  178.364495  0.0  0.0  33.315  271.8525   
4           8068.0   34.958440   78.381566  0.0  0.0   0.000   26.1725   
5           8064.0   62.382143  115.525501  0.0  0.0   0.000   66.9425   
6           8064.0  153.718605  192.928029  0.0  0.0  50.900  285.2700   
7           8064.0  119.855298  159.833026  0.0  0.0  23.290  214.8825   
8           8068.0   32.679751   69.232423  0.0  0.0   0.000   29.2350   
9           8064.0   73.006644  133.638656  0.0  0.0   0.000   81.0775   
10          8064.0  153.787966  191.442289  0.0  0.0  52.855  287.2750   
11          8064.0  115.541365  159.130603  0.0  0.0  16.730  201.8400   

                     Gb(i)              ...     load        residual_load  \
               max   count        mean  ...      75%    max         count   
dataset_id                              ...                                 
1           587.52  8064.0   43.221376  ...  217.125  395.0        8064.0   
2           631.02  8064.0  133.907484  ...  163.000  337.0        8064.0   
3           597.58  8064.0  127.242304  ...  179.000  320.0        8064.0   
4           412.32  8068.0   27.103291  ...  188.500  377.5        8068.0   
5           566.33  8064.0   50.093326  ...  211.500  372.0        8064.0   
6           664.03  8064.0  145.230279  ...  163.500  352.5        8064.0   
7           569.43  8064.0   87.447387  ...  175.500  328.5        8064.0   
8           401.58  8068.0   20.983778  ...  174.500  360.0        8068.0   
9           594.29  8064.0   62.369045  ...  175.500  354.0        8064.0   
10          659.34  8064.0  128.042011  ...  154.000  323.5        8064.0   
11          594.59  8064.0   96.795557  ...  169.500  311.0        8064.0   

                                                                              
                  mean         std     min       25%      50%    75%     max  
dataset_id                                                                    
1           120.885802   91.265376 -420.17  120.0000  135.500  154.5  330.05  
2            -6.380677  168.978928 -526.91 -107.7225   92.640  112.5  252.57  
3             4.614462  153.020519 -502.08  -88.8775   95.500  113.0  243.78  
4           120.884458   77.064032 -286.96  113.0000  123.500  142.5  321.96  
5           107.326873  105.299019 -442.49  111.5000  131.000  147.0  359.67  
6            -8.101231  173.444847 -517.43 -112.3875   94.320  116.5  284.11  
7            22.693003  136.943866 -462.04  -45.3825  101.135  114.5  253.55  
8           125.687750   67.339746 -204.22  117.5000  130.000  148.5  317.15  
9            80.548229  126.631301 -483.14   91.4325  121.000  133.0  312.94  
10          -17.015893  167.830975 -533.48 -122.2025   77.605  109.0  230.67  
11           20.808895  135.670368 -462.45  -39.6775   97.000  108.5  246.95  

[11 rows x 64 columns]

In [5]:
train['Year'] = pd.DatetimeIndex(train['time']).year  
train['Month'] = pd.DatetimeIndex(train['time']).month
train['Day'] = pd.DatetimeIndex(train['time']).day
train

,time,P,Gb(i),Gd(i),H_sun,T2m,WS10m,load,residual_load,dataset_id,Year,Month,Day
0,2018-01-01 00:00:00,0.0,0.0,0.0,0.0,9.11,5.38,113.5,113.5,1,2018,1,1
1,2018-01-01 00:15:00,0.0,0.0,0.0,0.0,8.89,5.43,119.0,119.0,1,2018,1,1
2,2018-01-01 00:30:00,0.0,0.0,0.0,0.0,8.66,5.48,117.5,117.5,1,2018,1,1
3,2018-01-01 00:45:00,0.0,0.0,0.0,0.0,8.44,5.54,114.5,114.5,1,2018,1,1
4,2018-01-01 01:00:00,0.0,0.0,0.0,0.0,8.22,5.59,123.0,123.0,1,2018,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88707,2020-10-10 22:45:00,0.0,0.0,0.0,0.0,6.49,2.83,99.5,99.5,11,2020,10,10
88708,2020-10-10 23:00:00,0.0,0.0,0.0,0.0,6.39,2.83,96.5,96.5,11,2020,10,10
88709,2020-10-10 23:15:00,0.0,0.0,0.0,0.0,6.34,2.83,100.5,100.5,11,2020,10,10
88710,2020-10-10 23:30:00,0.0,0.0,0.0,0.0,6.30,2.83,97.5,97.5,11,2020,10,10


In [6]:
train['Month'].dtype

dtype('int32')

In [7]:
# checking the test dataset
test

,time,Gb(i),Gd(i),H_sun,T2m,WS10m,dataset_id
0,2018-03-26 00:00:00,0.0,0.0,0.0,1.76,1.52,1
1,2018-03-26 00:15:00,0.0,0.0,0.0,1.57,1.52,1
2,2018-03-26 00:30:00,0.0,0.0,0.0,1.38,1.52,1
3,2018-03-26 00:45:00,0.0,0.0,0.0,1.19,1.52,1
4,2018-03-26 01:00:00,0.0,0.0,0.0,1.00,1.52,1
...,...,...,...,...,...,...,...
7387,2020-10-17 22:45:00,0.0,0.0,0.0,4.25,1.38,11
7388,2020-10-17 23:00:00,0.0,0.0,0.0,4.16,1.38,11
7389,2020-10-17 23:15:00,0.0,0.0,0.0,4.06,1.40,11
7390,2020-10-17 23:30:00,0.0,0.0,0.0,3.96,1.42,11


In [8]:
test.groupby('dataset_id').describe()

Gb(i)                                                       \
            count        mean         std  min  25%      50%       75%   
dataset_id                                                               
1           672.0   55.751324  136.909011  0.0  0.0    0.000   29.2075   
2           672.0  265.443542  306.132778  0.0  0.0  115.255  523.5550   
3           672.0   52.913676  127.378588  0.0  0.0    0.000   13.9750   
4           672.0    9.115685   40.844080  0.0  0.0    0.000    0.0000   
5           672.0   75.092500  161.494965  0.0  0.0    0.000   36.0550   
6           672.0  110.836220  203.439256  0.0  0.0    1.460  124.9575   
7           672.0   21.839077   67.585097  0.0  0.0    0.000    3.6200   
8           672.0    6.365000   25.152117  0.0  0.0    0.000    0.0000   
9           672.0  170.792158  239.793573  0.0  0.0   17.680  291.4325   
10          672.0  104.967798  191.220972  0.0  0.0    0.000  124.0575   
11          672.0   14.418676   46.759929  0.0  0.0    0.000    0.9450   

                    Gd(i)              ...      T2m         WS10m            \
               max  count        mean  ...      75%    max  count      mean   
dataset_id                             ...                                    
1           724.42  672.0   73.813810  ...   7.0800  12.08  672.0  2.406280   
2           965.60  672.0   83.219330  ...  21.7300  25.91  672.0  2.519449   
3           533.26  672.0   54.581964  ...  11.7075  15.96  672.0  2.565655   
4           331.37  672.0   15.988095  ...   5.2725   7.51  672.0  3.513631   
5           677.85  672.0   72.213512  ...  11.9200  16.29  672.0  2.139033   
6           797.71  672.0  118.150000  ...  18.1650  24.90  672.0  2.367024   
7           485.77  672.0   49.973393  ...  11.5700  15.82  672.0  3.443527   
8           265.38  672.0   17.575923  ...   6.6700  10.00  672.0  3.381101   
9           737.76  672.0   77.797485  ...  14.2475  20.23  672.0  2.120149   
10          905.83  672.0  106.575164  ...  17.6125  23.13  672.0  1.689792   
11          311.12  672.0   44.863229  ...   8.0100  11.42  672.0  2.485327   

                                                       
                 std   min    25%   50%     75%   max  
dataset_id                                             
1           1.053523  0.28  1.520  2.34  3.2200  5.59  
2           1.035300  0.14  1.930  2.55  3.1950  4.41  
3           1.199240  0.48  1.660  1.95  3.5200  5.59  
4           1.421390  1.03  2.210  3.69  4.5500  6.34  
5           0.808576  0.14  1.660  2.07  2.5500  4.28  
6           1.043150  0.55  1.520  2.00  3.3100  4.76  
7           1.075494  1.10  2.800  3.49  4.2850  6.76  
8           0.838240  1.38  2.685  3.49  4.0900  5.10  
9           1.006901  0.21  1.450  1.93  2.5225  4.83  
10          1.088400  0.07  1.030  1.45  2.1400  5.59  
11          1.004848  0.76  1.720  2.38  3.2400  5.31  

[11 rows x 40 columns]

In [9]:
test['Year'] = pd.DatetimeIndex(test['time']).year  
test['Month'] = pd.DatetimeIndex(test['time']).month
test['Day'] = pd.DatetimeIndex(test['time']).day
test

,time,Gb(i),Gd(i),H_sun,T2m,WS10m,dataset_id,Year,Month,Day
0,2018-03-26 00:00:00,0.0,0.0,0.0,1.76,1.52,1,2018,3,26
1,2018-03-26 00:15:00,0.0,0.0,0.0,1.57,1.52,1,2018,3,26
2,2018-03-26 00:30:00,0.0,0.0,0.0,1.38,1.52,1,2018,3,26
3,2018-03-26 00:45:00,0.0,0.0,0.0,1.19,1.52,1,2018,3,26
4,2018-03-26 01:00:00,0.0,0.0,0.0,1.00,1.52,1,2018,3,26
...,...,...,...,...,...,...,...,...,...,...
7387,2020-10-17 22:45:00,0.0,0.0,0.0,4.25,1.38,11,2020,10,17
7388,2020-10-17 23:00:00,0.0,0.0,0.0,4.16,1.38,11,2020,10,17
7389,2020-10-17 23:15:00,0.0,0.0,0.0,4.06,1.40,11,2020,10,17
7390,2020-10-17 23:30:00,0.0,0.0,0.0,3.96,1.42,11,2020,10,17


In [10]:
train.isna().sum()

time             0
P                0
Gb(i)            0
Gd(i)            0
H_sun            0
T2m              0
WS10m            0
load             0
residual_load    0
dataset_id       0
Year             0
Month            0
Day              0
dtype: int64

In [11]:
test.isna().sum()

time          0
Gb(i)         0
Gd(i)         0
H_sun         0
T2m           0
WS10m         0
dataset_id    0
Year          0
Month         0
Day           0
dtype: int64

In [12]:
y = train.residual_load
feature_columns = ['Gb(i)', 'Gd(i)', 'H_sun', 'T2m', 'WS10m', 'Year', 'Month', 'Day']
X = train[feature_columns]
X_test = test[feature_columns]

In [13]:
# fill in and uncomment
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

In [14]:
# Specify the DecisionTreeRegressor model
dtr_model = DecisionTreeRegressor(random_state=1)

# Fit iowa_model with the training data.
dtr_model.fit(train_X, train_y)

DecisionTreeRegressor(random_state=1)

In [15]:
val_predictions = dtr_model.predict(val_X)

# print the top few validation predictions
print(val_predictions)
# print the top few actual prices from validation data
print(val_y)

[ 224.95  207.4   129.5  ...  135.5   -21.33 -349.1 ]
2049     252.02
2058     178.49
61237    135.00
445      168.00
12734    135.54
          ...  
70897   -167.87
34467    140.00
5006     132.00
20606    -53.81
82607   -368.31
Name: residual_load, Length: 17743, dtype: float64


In [16]:
val_mae = mean_absolute_error(val_y, val_predictions)

# uncomment following line to see the validation_mae
print(val_mae)

27.93993274342933


In [17]:
# Specify the RandomForestRegressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)

# Fit iowa_model with the training data.
rf_model.fit(train_X, train_y)

RandomForestRegressor(random_state=0)

In [18]:
rf_val_predictions = rf_model.predict(val_X)

# print the top few validation predictions
print(rf_val_predictions)

[ 210.4416      185.2956      154.89910119 ...  133.55        -71.5992
 -334.8359    ]


In [19]:
rf_val_mae = mean_absolute_error(val_y, rf_val_predictions)

# uncomment following line to see the validation_mae
print(rf_val_mae)

21.559070935486297


In [20]:
# Specify the XGBRegressor model
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.3)

# Fit iowa_model with the training data.
xgb_model.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [21]:
xgb_val_predictions = xgb_model.predict(val_X)

# print the top few validation predictions
print(xgb_val_predictions)

[ 197.09727  198.8996   135.98203 ...  137.9249   -68.82535 -367.31107]


In [22]:
xgb_val_mae = mean_absolute_error(val_y, xgb_val_predictions)

# uncomment following line to see the validation_mae
print(xgb_val_mae)

19.914704722031335


In [23]:
test['residual_load'] = xgb_model.predict(X_test)

# print the top few validation predictions
output = test[['time','residual_load']]
output.to_csv('submission.csv', index=False)